# Quantização GGUF com llama.cpp - cpu only

## Login - Hugginface Hub


In [22]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [20]:
cache_dir = ''

In [ ]:
# from google.colab import drive 
# drive.mount('/content/drive')

In [ ]:
# import os
# cache_dir = "/content/drive/My Drive/huggingface_cache"
# os.makedirs(cache_dir, exist_ok=True) 

In [ ]:
# ! pip install -q einops
# ! pip install sentencepiece
# ! pip install gguf>=0.1.0
# ! pip install protobuf>=4.21.0

## Imports


In [1]:
import os
#os.environ ["CUDA_VISIBLE_DEVICES" ]="0"
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

## Model loading


In [34]:
model_name = 'mistralai/Mistral-7B-v0.1'

model = AutoModelForCausalLM. from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map='cpu', # ou 'auto', não importa
    offload_folder='offload',
    cache_dir=cache_dir
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
! git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...


In [12]:
%cd llama.cpp 
# %cd ../
# Resposta: /workspace/llama.cpp

D:\Capgemini\LLMS\Quantizacao\llama.cpp


In [ ]:
model.save_pretrained('./models/')

## Download dos tokenizer configs


In [37]:
import os
import requests

def download_file_from_huggingface(model_name, filename, save_path, token):
    headers = {"Authorization": f"Bearer {token}"}
    url = f"https://huggingface.co/{model_name}/resolve/main/{filename}"
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        print(f"Failed to download {filename}. HTTP Status Code: {r.status_code}") 
        return False
    with open(os.path.join(save_path, filename), 'wb') as f:
        f.write(r.content)
    return True

def main():
    # HuggingFace token
    huggingface_token = "SEU_token"

    files_to_download = [
        "tokenizer_config.json",
        "tokenizer.model",
        "tokenizer.json",
        "special_tokens_map.json",
    ]

    # Nome/path do modelo
    model_name = 'mistralai/Mistral-7B-v0.1'

    # Criar diretório de modelos 
    save_path = "./models"
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Download files - usuário logado
    for filename in files_to_download:
        success = download_file_from_huggingface(model_name, filename, save_path, huggingface_token)
        if success:
            print(f"Successfully downloaded {filename}")
        else:
            print(f"Failed to download {filename}")

if __name__ == "__main__":
    main()


Successfully downloaded tokenizer_config.json
Successfully downloaded tokenizer.model
Successfully downloaded tokenizer.json
Successfully downloaded special_tokens_map.json


## Conversão para GGUF


In [ ]:
# Tem de ser em Linux o WSL

# Debian-based Linux padrão
# !apt update -y
# !apt install build-essential git cmake libopenblas-dev libeigen3-dev -y

# Ou (CentOS 7 do BB)
# !sudo yum groupinstall 'Development Tools' -y
# !sudo yum install git cmake openblas-devel eigen3-devel -y

In [ ]:
!make LLAMA_OPENBLAS=1

In [30]:
!python convert.py models/

Loading model file models\model.safetensors
params = Params(n_vocab=32003, n_embd=2048, n_layer=22, n_ctx=2048, n_ff=5632, n_head=32, n_head_kv=4, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=10000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=None, path_model=WindowsPath('models'))
32000 32000
Vocab info: <VocabLoader with 32000 base tokens and 3 added tokens>
Special vocab info: <SpecialVocab with 61249 merges, special tokens {'bos': 1, 'eos': 2, 'unk': 0, 'pad': 32000}, add special tokens unset>
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting layer 12
Permuting layer 13
Permuting layer 14
Permuting layer 15
Permuting layer 16
Permuting layer 17
Permuting layer 18
Permuting layer 19
Permuting layer 20
Permuting layer 21
lm_head.weight   

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# GGML está DEPRECATED!
#!./quantize ./ggml-model-f16.bin ./ggml-model-q4_K_M.bin q4_K_M

In [ ]:
# Split string pelo caracter '/'
parts = model_name.split('/')

# Pegar a segund aparte (index 1) 
model_name_pure = parts[1]

quant_type = "Q4_K"
quantized_model = f'models/{model_name_pure}.{quant_type}.gguf'
print(f'Preparando {quantized_model} com {quant_type} quantização.')

import subprocess

# Build o command como uma lista argumentos
command = ["./quantize", "models/ggml-model-f16.gguf", quantized_model, quant_type]

# Executar o command
subprocess.run(command)